In [2]:
from Sample_Gibbs_from_Gaussian import log_P
from Sample_Gibbs_from_Gaussian import sample_w
from tqdm import tqdm
import numpy as np
sample_size = 100000
p_current = 4
beta = 10
sigma_q = 0.5
################################################################
original_random_state = np.random.get_state()
np.random.seed(20230929)
#gaussian distribution sampler: normal(mean=0.0, variance_sqrt=1.0, size=None) s = np.random.normal(0,1)
#teacher model
#dimension of x: d（固定不变）10->5
d = 5
#number of training samples（固定不变）100->20
N = 20
#dimension of hypothesis space
p0 = 5 #dimension of teacher model（固定不变）
p = p_current #dimension of student model（可变）#p=200
p_max = 2000 #upper bound of dimension of student model（固定不变）
#point-wise activate function f:tanh
#variance of random noise added to y
sigma = 0.1

#random feature matrix
F0 = np.random.normal(0,1,(d,p0)) #生成teacher model的random feature matrix F0（固定不变）
F = np.random.normal(0,1,(d,p_max))[:,:p] #生成student model的random feature matrix F(d*p)。先按照p_max的大小生成，再根据当前维度p的不同截取前p列形成每个p对应的F。
    #注意，这个F对于一般的神经网络是可训练的，但是对于RFM为了简化直接设定为固定值。（固定不变）
#teacher parameter w(p) with lambda = ? until each dim of Y~1e0
lambda_ = 0.0001
#w_0 = np.random.normal(0,sigma/np.sqrt(lambda_*N),p)
w_0 = np.random.normal(0,1,p0) #生成teacher model的权重向量（固定不变）
#w_0 = np.concatenate((w_0,np.zeros(5)),axis = 0)

np.random.set_state(original_random_state)
################################################################

w = []
B = []
Y = []
for i in tqdm(range(sample_size)):
    X = np.random.normal(0,1,(N,d))
    X_rf0 = np.tanh(X.dot(F0)/np.sqrt(d))
    Y_pure = X_rf0.dot(w_0)
    Y_i = Y_pure + np.random.normal(0,sigma,N) #由X通过teacher model（也就是F0和w_0）生成的Y
    #S_i = np.concatenate((X,Y_i.reshape(N,-1)),axis=1).reshape(-1)  #S=(X,Y)[0]
    B_i = np.tanh(X.dot(F)/np.sqrt(d))
    w_i = sample_w(p,N,10,0.5,B_i,Y_i,1)[0]
    w.append(w_i)
    B.append(B_i)
    Y.append(Y_i)
w_shuffle = np.random.permutation(w)
skl = 0
for i in tqdm(range(len(w))):
    skl+=log_P(w[i],B[i],Y[i],N,beta,sigma_q)-log_P(w_shuffle[i],B[i],Y[i],N,beta,sigma_q)
skl /= len(w)
skl

  0%|          | 0/100000 [00:00<?, ?it/s]

100%|██████████| 100000/100000 [00:26<00:00, 3741.45it/s]


0.4810719675338408